J1J2 model

In [1]:
import quimb.tensor as qtn
import quimb as qu

import netket as nk
from netket.graph import Lattice
from vmc_torch.hamiltonian import spin_Heisenberg_square_lattice
import numpy as np
from math import pi
from autoray import do

ndim = 2
Lx = 4
Ly = 4
L = 2
pbc = False
total_sz = 0.0
print(f"Total Sz = {total_sz}")
D = 2

basis = np.array([
     [1.0,0.0],
     [0.0,1.0],
 ])
custom_edges = [
     (0, 0, [1.0,0.0], 0),
     (0, 0, [0.0,1.0], 0),
     (0, 0, [1.0, 1.0], 1),
     (0, 0, [1.0, -1.0], 1),
 ]

g = Lattice(basis_vectors=basis, pbc=False, extent=[Lx, Ly],
     custom_edges=custom_edges)

n = g.n_nodes
hi = nk.hilbert.Spin(s=1/2, total_sz=0.0, N=n)
# Heisenberg with coupling J=1.0 for nearest neighbors
# and J=0.5 for next-nearest neighbors
H = nk.operator.Heisenberg(hilbert=hi, graph=g, J=(1.0,0.5)) # In Netket, the spin operators are Pauli matrices, while in Quimb they are 1/2*Pauli matrices
# ham0 = spin_Heisenberg_square_lattice(Lx, Ly, J=1.0, pbc=pbc,total_sz=total_sz)
# hi0 = ham0.hilbert

/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total Sz = 0.0


In [2]:
# compute the ground-state energy (here we only need the lowest energy, and do not need the eigenstate)
evals = nk.exact.lanczos_ed(H, compute_eigenvectors=False)
exact_gs_energy = evals[0]
print('The exact ground-state energy is E0=',exact_gs_energy)
print(exact_gs_energy/(Lx*Ly)/4)

The exact ground-state energy is E0= -30.02222780032426
-0.46909730938006655


In [3]:
# PEPS tensor network
psi = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=D, phys_dim=2) # initialization from PEPS
J1=1.0
J2=0.5
# ham = qtn.ham_2d_heis(Lx=Lx, Ly=Ly, j=4*J)
ham = qtn.ham_2d_j1j2(Lx, Ly, j1=J1, j2=J2)
# su = qtn.tensor_arbgeom_tebd.SimpleUpdateGen(
#     psi, 
#     ham,
#     compute_energy_every=10,
#     compute_energy_per_site=True,
# )

su = qtn.tensor_2d_tebd.SimpleUpdate(
    psi, 
    ham,
    compute_energy_every=10,
    compute_energy_per_site=True,
)

for tau in [1.0, 0.3, 0.1, 0.03, 0.01]:
    su.evolve(100, tau=tau)
psi_su = su.state

peps = su.get_state()
peps.equalize_norms_(value=1)

# save the state
params, skeleton = qtn.pack(peps)

import os
import pickle
os.makedirs(f'../../data/{Lx}x{Ly}/J1={J1}_J2={J2}/D={D}', exist_ok=True)

with open(f'../../data/{Lx}x{Ly}/J1={J1}_J2={J2}/D={D}/peps_skeleton.pkl', 'wb') as f:
    pickle.dump(skeleton, f)
with open(f'../../data/{Lx}x{Ly}/J1={J1}_J2={J2}/D={D}/peps_su_params.pkl', 'wb') as f:
    pickle.dump(params, f)

n=500, tau=0.0100, energy~-0.427732: 100%|##########| 100/100 [00:03<00:00, 29.56it/s]


Heisenberg model

In [36]:
import quimb.tensor as qtn
import quimb as qu

import netket as nk
from vmc_torch.hamiltonian import spin_Heisenberg_square_lattice
import numpy as np
from math import pi
from autoray import do

ndim = 2
Lx = 2
Ly = 2
L = 2
pbc = False
total_sz = 0.0
print(f"Total Sz = {total_sz}")
D = 4

# Build square lattice with nearest and next-nearest neighbor edges
# lattice = nk.graph.Square(L, max_neighbor_order=1, pbc=False)
# g = lattice = nk.graph.Hypercube(L, ndim, pbc=pbc)
g = lattice = nk.graph.Grid([Lx, Ly], pbc=pbc)
# g = lattice = nk.graph.Pyrochlore([L, L, L], pbc=pbc)

n = lattice.n_nodes
hi = nk.hilbert.Spin(s=1 / 2, total_sz=0.0, N=n)
# Heisenberg with coupling J=1.0 for nearest neighbors
# and J=0.5 for next-nearest neighbors
# H = nk.operator.Ising(hilbert=hi, graph=lattice, J=1.0, h=1.0)
H = nk.operator.Heisenberg(hilbert=hi, graph=lattice, J=1.0, sign_rule=False) # In Netket, the spin operators are Pauli matrices, while in Quimb they are 1/2*Pauli matrices

ham0 = spin_Heisenberg_square_lattice(Lx, Ly, J=1.0, pbc=pbc,total_sz=total_sz)
hi0 = ham0.hilbert

Total Sz = 0.0


In [37]:
# compute the ground-state energy (here we only need the lowest energy, and do not need the eigenstate)
evals = nk.exact.lanczos_ed(H, compute_eigenvectors=False)
exact_gs_energy = evals[0]
print('The exact ground-state energy is E0=',exact_gs_energy)
print(exact_gs_energy/(Lx*Ly)/4)

The exact ground-state energy is E0= -7.999999999999999
-0.49999999999999994


In [38]:
# PEPS tensor network
psi = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=D, phys_dim=2) # initialization from PEPS
J=1.0
ham = qtn.ham_2d_heis(Lx=Lx, Ly=Ly, j=J)
su = qtn.tensor_arbgeom_tebd.SimpleUpdateGen(
    psi, 
    ham,
    compute_energy_every=10,
    compute_energy_per_site=True,
)
for tau in [1.0, 0.3, 0.1, 0.03, 0.01]:
    su.evolve(100, tau=tau)
psi_su = su.state

# peps = su.get_state()
# peps.equalize_norms_(value=1)

# # save the state
# params, skeleton = qtn.pack(peps)

# import os
# import pickle
# os.makedirs(f'../../data/{Lx}x{Ly}/J={J}/D={D}', exist_ok=True)

# with open(f'../../data/{Lx}x{Ly}/J={J}/D={D}/peps_skeleton.pkl', 'wb') as f:
#     pickle.dump(skeleton, f)
# with open(f'../../data/{Lx}x{Ly}/J={J}/D={D}/peps_su_params.pkl', 'wb') as f:
#     pickle.dump(params, f)
    

n=0, tau=0.0000, energy~-0.030482:   0%|          | 0/100 [00:00<?, ?it/s]

n=500, tau=0.0100, energy~-0.439450: 100%|##########| 100/100 [00:00<00:00, 113.96it/s]


In [189]:
def from_netket_to_quimb_spin_config(config):
    """From (-1,1) to (0,1) basis"""
    def func(x):
        return do('array',(x + 1) / 2, dtype=int)
    if len(config.shape) == 1:
        return func(config)
    else:
        return np.array([func(c) for c in config])

def get_amp(psi, config):
    return psi.isel({psi.site_inds[i]: int(s) for i, s in enumerate(config)})

def to_array(psi):
    return psi.to_dense()

def to_array_amps(psi, configs_netket):
    configs = from_netket_to_quimb_spin_config(configs_netket)
    return np.array([get_amp(psi, config).contract() for config in configs])

def to_array_config_dict(psi, configs_netket):
    configs = from_netket_to_quimb_spin_config(configs_netket)
    return {tuple(config): get_amp(psi, config).contract() for config in configs}

In [190]:
psi_vec = to_array_amps(psi_su, hi.all_states())
# psi_dict = to_array_config_dict(psi_su, hi.all_states())

In [192]:
hamiltonian = H.to_dense()

print("Energy from Quimb:", np.vdot(psi_vec, hamiltonian @ psi_vec).real / np.vdot(psi_vec, psi_vec).real/(Lx*Ly)/4)

Energy from Quimb: -0.528558043149109


In [193]:
# psi_su.view_as_(qtn.PEPS)
psi_su.compute_local_expectation(ham.terms, normalized=True, max_bond=128)/(Lx*Ly)/4

-0.49680707385756656

In [153]:
import numpy as np
from quimb.utils import progbar as Progbar
from mpi4py import MPI
import pickle
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

# torch
from torch.nn.parameter import Parameter
import torch
import torch.nn as nn
torch.autograd.set_detect_anomaly(False)

# quimb
import quimb as qu
import quimb.tensor as qtn
import autoray as ar
from autoray import do

from vmc_torch.experiment.tn_model import PEPS_model, PEPS_NN_Model, init_weights_to_zero, PEPS_NNproj_Model
from vmc_torch.sampler import MetropolisExchangeSamplerSpinless, MetropolisExchangeSamplerSpinful
from vmc_torch.variational_state import Variational_State
from vmc_torch.optimizer import TrivialPreconditioner, SignedSGD, SGD, SR
from vmc_torch.VMC import VMC
from vmc_torch.hamiltonian import square_lattice_spin_Heisenberg, spin_Heisenberg_square_lattice
from vmc_torch.torch_utils import SVD,QR

# Register safe SVD and QR functions to torch
ar.register_function('torch','linalg.svd',SVD.apply)
ar.register_function('torch','linalg.qr',QR.apply)

from vmc_torch.global_var import DEBUG


COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

# Hamiltonian parameters
J = 1.0
# H, hi, graph = square_lattice_spinful_Fermi_Hubbard(Lx, Ly, t, U, N_f)
H = spin_Heisenberg_square_lattice(L, J, total_sz=0.0)
graph = H.graph
# TN parameters
D = 2
chi = -1
dtype=torch.float64

# Load PEPS
skeleton = pickle.load(open(f"../../data/{L}x{L}/J={J}/D={D}/peps_skeleton.pkl", "rb"))
peps_params = pickle.load(open(f"../../data/{L}x{L}/J={J}/D={D}/peps_su_params.pkl", "rb"))
peps = qtn.unpack(peps_params, skeleton)
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))

# VMC sample size
N_samples = 2**10
N_samples = N_samples - N_samples % SIZE + SIZE
if (N_samples/SIZE)%2 != 0:
    N_samples += SIZE

model = PEPS_model(peps, max_bond=None)
# model = PEPS_NN_Model(peps, max_bond=chi, nn_eta=1.0, nn_hidden_dim=L**2)
# model = PEPS_NNproj_Model(peps, max_bond=chi, nn_eta=1.0, nn_hidden_dim=L**2)
# model.apply(init_weights_to_zero)
model_names = {
    PEPS_model: 'PEPS',
    PEPS_NN_Model: 'PEPS_NN',
    PEPS_NNproj_Model: 'PEPS_NNproj'
}
model_name = model_names.get(type(model), 'UnknownModel')

init_step = 0
total_steps = 200
if init_step != 0:
    saved_model_params = torch.load(f'../../data/{L}x{L}/J={J}/D={D}/{model_name}/chi={chi}/model_params_step{init_step}.pth')
    saved_model_state_dict = saved_model_params['model_state_dict']
    saved_model_params_vec = torch.tensor(saved_model_params['model_params_vec'])
    try:
        model.load_state_dict(saved_model_state_dict)
    except:
        model.load_params(saved_model_params_vec)

# optimizer = SignedSGD(learning_rate=0.05)
optimizer = SGD(learning_rate=0.05)
sampler = MetropolisExchangeSamplerSpinless(H.hilbert, graph, N_samples=N_samples, burn_in_steps=20, reset_chain=False, random_edge=False, dtype=dtype)
variational_state = Variational_State(model, hi=H.hilbert, sampler=sampler, dtype=dtype)
preconditioner = SR(dense=False, exact=True if sampler is None else False, use_MPI4Solver=True, diag_eta=0.05, iter_step=1e5, dtype=dtype)
vmc = VMC(H, variational_state, optimizer, preconditioner)


In [161]:
batchsize = 500
# samples = sampler.sample(H, variational_state, chain_length=batchsize)
# samples[0]/batchsize/L**2/4
samples = sampler.sample_expectation(variational_state, H, chain_length=batchsize)

In [170]:
O_loc = np.asarray(samples[1])/L**2/4
samples[0]/L**2/4

np.float64(-0.49400048802251967)